In [54]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

conn = sqlite3.connect("../db/baltasar.db")
def sql(query):
    #print(query)
    return pd.read_sql_query(query, conn)

In [113]:
dimensions = ['type', 'operator', 'age', 'activeness', 'yield', 'parents', 'children', 'holdingSize', 'numberOfTokens']

def getTokenPrices(startDate, endDate):
    return sql('''SELECT
                endPrice.token, 
                CASE
                    WHEN endPrice.price / prevPrice.price > 1.5 THEN "FLUC"
                    WHEN endPrice.price / prevPrice.price <= 0.67 THEN "FLUC"
                    ELSE "ORDINARY"
                END as success,
                endPrice.price / prevPrice.price as valueChange
            FROM
                Price endPrice
                INNER JOIN
                Price prevPrice
                ON endPrice.token = prevPrice.token
            WHERE
                prevPrice.date='''+str(taxonomyDate)+''' and
                endPrice.date=''' + str(endDate))
               
def getOwnershipDistributionPerDimension(dimension, table='currentTaxonomyResults'):
    grouped = sql('''SELECT 
        '''+dimension+ ''' as dimension,
        token,
        sum(percTokens) as percToken
        FROM
            '''+table+'''
        GROUP BY token, '''+dimension
       )
    grouped['dimension'] = grouped['dimension'].apply(lambda c: dimension[:2]+"_"+c)
    grouped = pd.pivot_table(grouped, values='percToken', index=['token'],
                     columns=['dimension'], aggfunc=np.sum)
    return grouped

def makeDataSet(table, startDate, endDate):
    tokens = getTokenPrices(startDate, endDate)
    tokens = tokens.set_index('token')
    # single columns
    for dimension in dimensions:
        grouped = getOwnershipDistributionPerDimension(dimension, table)
        tokens = tokens.join(grouped)
    return tokens

## Times

# Train Random Forest

In [89]:
taxonomyDate = int(float(sql('''
SELECT timestamp FROM Etransfer
WHERE blocknumber=(SELECT max(blocknumber) FROM EntityTaxonomy)
LIMIT 1''')['timestamp'][0]) / 86400) * 86400
endDate = sql('''SELECT max(date) as d FROM Price''')['d'][0]

trainDate_start = int(float(sql('''SELECT timestamp FROM ETransfer WHERE blocknumber=7020000''')['timestamp'][0]) / 86400) * 86400
trainDate_end = trainDate_start + 118 * 86400

In [180]:
# train = makeDataSet('FormerTaxonomyResults', trainDate_start, trainDate_end)
train['success'] = 'ORDINARY'
train.loc[train['valueChange'] < train['valueChange'].quantile(0.25), 'success'] = 'FLUC'
train.loc[train['valueChange'] >= train['valueChange'].quantile(0.75), 'success'] = 'FLUC'


In [3]:
'''
for (i in range(len(dimensions)):
    for (j in range(len(i,dimensions)):
        dimensionA = dimensions[i]
        dimensionB = dimensions[j]
        if dimensionA == dimensionB:
            continue
        grouped = data[['token', dimensionA, dimensionB,'percToken']].groupby(['token', dimensionA, dimensionB]).sum().reset_index()

'''

,type,operator,age,activeness,yield,parents,children,holdingSize,numberOfTokens,token,numAddresses,percTokens
0,concentrator,capp,over1Year,daily,loss<.9,diverse,diverse,over_1kkUSD,>30,0xbtc,1,2.450236e-10


In [193]:
labels = np.array(train['success'])
y = np.array(labels)
train = train.fillna(0)
features = train.drop(['success','valueChange'], axis = 1)
feature_list = list(features.columns)
features = np.array(features)
scaler = StandardScaler()
#data = tokens.values()
scaler.fit(features)
X = scaler.transform(features)
clf = RandomForestClassifier(n_estimators=10000,
                              max_depth=None, min_samples_split=2, max_features="sqrt")
clf.fit(X, y) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [194]:
# Get numerical feature importances
importances = list(clf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: ty_connector_complex Importance: 0.02
Variable: ty_connector_simple  Importance: 0.02
Variable: ty_exchange          Importance: 0.02
Variable: ty_mixer             Importance: 0.02
Variable: ty_sink_simple       Importance: 0.02
Variable: op_capp              Importance: 0.02
Variable: op_other             Importance: 0.02
Variable: ag_month             Importance: 0.02
Variable: ag_quarter           Importance: 0.02
Variable: ag_week              Importance: 0.02
Variable: ag_year              Importance: 0.02
Variable: ac_daily             Importance: 0.02
Variable: ac_monthly           Importance: 0.02
Variable: yi_loss<.67          Importance: 0.02
Variable: yi_loss<.9           Importance: 0.02
Variable: yi_profit<1.5        Importance: 0.02
Variable: yi_profit<10         Importance: 0.02
Variable: yi_steady>=.9,<1.1   Importance: 0.02
Variable: pa_connector_complex Importance: 0.02
Variable: pa_connector_simple  Importance: 0.02
Variable: pa_dilluter          Importanc

# Testing

In [195]:
#test = makeDataSet('CurrentTaxonomyResults', taxonomyDate, endDate)
test['success'] = 'ORDINARY'

test.loc[test['valueChange'] < test['valueChange'].quantile(0.25), 'success'] = 'FLUC'
test.loc[test['valueChange'] >= test['valueChange'].quantile(0.75), 'success'] = 'FLUC'

In [196]:
labels = np.array(test['success'])
y = np.array(labels)
train = train.fillna(0)
features = train.drop(['success','valueChange'], axis = 1)
feature_list = list(features.columns)
features= np.array(features)
X_test = scaler.transform(features)



test['prediction_proba'] = clf.predict_proba(X_test)[:,0]
test['prediction'] = clf.predict(X_test)
print(test[['success', 'prediction', 'prediction_proba']].sort_values(by='prediction_proba'))

#from sklearn.metrics import confusion_matrix
confusion_matrix(np.array(test['prediction']), labels)

                             success prediction  prediction_proba
token                                                            
sharpay                     ORDINARY   ORDINARY            0.0722
hyperion                        FLUC   ORDINARY            0.0808
xovbank                         FLUC   ORDINARY            0.0894
acute-angle-cloud           ORDINARY   ORDINARY            0.0950
stockchain                      FLUC   ORDINARY            0.0953
decentraland                ORDINARY   ORDINARY            0.0975
kucoin-shares                   FLUC   ORDINARY            0.0991
jibrel-network              ORDINARY   ORDINARY            0.1078
iostoken                    ORDINARY   ORDINARY            0.1098
optitoken                   ORDINARY   ORDINARY            0.1103
streamr-datacoin            ORDINARY   ORDINARY            0.1108
smartmesh                   ORDINARY   ORDINARY            0.1111
ubcoin-market               ORDINARY   ORDINARY            0.1123
kyber-netw

array([[248, 239],
       [239, 247]])

In [198]:
from scipy.stats import chi2_contingency 

#test_safe = test

obs = confusion_matrix(np.array(test['prediction']), np.array(test['success']))
#print(test_safe[['prediction', 'success', 'prediction_proba']])
print(chi2_contingency(obs))
print(obs)




(0.23121237804299072, 0.6306263220153304, 1, array([[243.75025694, 243.24974306],
       [243.24974306, 242.75025694]]))
[[248 239]
 [239 247]]
